# Projekat - Predviđanje srčane bolesti

### Autor - Nenad Radović, RA18/2020

## Modeli i predikcija

### Učitavanje preprocesiranih podataka

Učitajmo prethodno procesirane podatke. Podaci dolaze u formi trening i test skupova za svaki od modela, te nije potrebno naknadno dijeliti ih.

In [1]:
import pandas as pd
import os

# Logistička regresija
X_train_lr = pd.read_csv('Logistic_Regression/X_train_lr.csv')
y_train_lr = pd.read_csv('Logistic_Regression/y_train_lr.csv')
X_test_lr = pd.read_csv('Logistic_Regression/X_test_lr.csv')
y_test_lr = pd.read_csv('Logistic_Regression/y_test_lr.csv')

# KNN i model stabla odluke
X_train_kt = pd.read_csv('KNN_DT/X_train_kt.csv')
y_train_kt = pd.read_csv('KNN_DT/y_train_kt.csv')
X_test_kt = pd.read_csv('KNN_DT/X_test_kt.csv')
y_test_kt = pd.read_csv('KNN_DT/y_test_kt.csv')

### Modeli

#### Logistička regresija

Prvi model koji ćemo razmotriti jeste model logističke regresije. Učitajmo ga, *fit*-ujmo model i uradimo predikciju na *test* skupu.

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

model_lr = LogisticRegression()
model_knn = KNeighborsClassifier()
model_dt = DecisionTreeClassifier()

Bacimo se na podešavanje i pronalaženje optimalnih hiperparametara za naše modele.

In [3]:
# Hiperparametri logističke regresije
from sklearn.model_selection import GridSearchCV

params_lr = [{
    'C': [i/10 for i in range(1, 13)],
    'tol': [i/100000 for i in range(1, 13)],
    'solver': ['newton-cholesky', 'lbfgs']
}]

gs_lr = GridSearchCV(estimator=model_lr, param_grid=params_lr, cv=9).fit(X=X_train_lr.to_numpy(), y=y_train_lr.to_numpy().ravel())

Najbolji parametri logističke regresije su

In [4]:
gs_lr.best_params_

{'C': 0.3, 'solver': 'newton-cholesky', 'tol': 1e-05}

Sada smo našli optimalne parametre modela, te ćemo ga istrenirati i predvidjeti *target*-e na test skupu.

In [5]:
model_lr.C = gs_lr.best_params_['C']
model_lr.tol = gs_lr.best_params_['tol']
model_lr.solver = gs_lr.best_params_['solver']

model_lr.fit(X=X_train_lr.to_numpy(), y=y_train_lr.to_numpy().ravel())
y_pred_lr = model_lr.predict(X=X_test_lr.to_numpy())

Model je sada kreiran i istreniran. Ispišimo neke od karakteristika modela - tačnost, preciznost, odziv, F1 rezultat, kao i matricu konfuzije. Naredne performanse će imati sufiks '*_bf*', što označava su performanse prije smanjivanja dimenzionalnosti modela, odnosno izabiranja najbitnijih *feature* kolona.

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

as_lr_bf = accuracy_score(y_true=y_test_lr.to_numpy().ravel(), y_pred=y_pred_lr)
as_lr_bf

0.8705357142857143

In [7]:
ps_lr_bf = precision_score(y_true=y_test_lr.to_numpy().ravel(), y_pred=y_pred_lr)
ps_lr_bf

0.875

In [8]:
rs_lr_bf = recall_score(y_true=y_test_lr.to_numpy().ravel(), y_pred=y_pred_lr)
rs_lr_bf

0.8504672897196262

In [9]:
f1s_lr_bf = f1_score(y_true=y_test_lr.to_numpy().ravel(), y_pred=y_pred_lr)
f1s_lr_bf

0.8625592417061612

In [10]:
cms_lr_bf = confusion_matrix(y_true=y_test_lr.to_numpy().ravel(), y_pred=y_pred_lr)
cms_lr_bf

array([[104,  13],
       [ 16,  91]], dtype=int64)

Uradimo iste postupke na modelu KNN, kao i na stablu odlučivanja.

#### KNN model

In [11]:
# Hiperparametri KNN modela
params_knn = [{
    'n_neighbors': [2*i+1 for i in range(2, 12)],
    'weights': ['uniform', 'distance'],
    'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'metric': ['euclidean', 'cityblock', 'manhattan']
}]

gs_knn = GridSearchCV(estimator=model_knn, param_grid=params_knn, cv=6).fit(X=X_train_kt.to_numpy(), y=y_train_kt.to_numpy().ravel())

Najbolji parametri KNN modela su

In [12]:
gs_knn.best_params_

{'algorithm': 'ball_tree',
 'metric': 'cityblock',
 'n_neighbors': 5,
 'weights': 'distance'}

In [13]:
model_knn.n_neighbors = gs_knn.best_params_['n_neighbors']
model_knn.weights = gs_knn.best_params_['weights']
model_knn.algorithm = gs_knn.best_params_['algorithm']
model_knn.metric = gs_knn.best_params_['metric']

model_knn.fit(X=X_train_kt.to_numpy(), y=y_train_kt.to_numpy().ravel())
y_pred_knn = model_knn.predict(X=X_test_kt.to_numpy())

In [14]:
as_knn_bf = accuracy_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=y_pred_knn)
as_knn_bf

0.8392857142857143

In [15]:
ps_knn_bf = precision_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=y_pred_knn)
ps_knn_bf

0.8256880733944955

In [16]:
rs_knn_bf = recall_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=y_pred_knn)
rs_knn_bf

0.8411214953271028

In [17]:
f1s_knn_bf = f1_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=y_pred_knn)
f1s_knn_bf

0.8333333333333334

In [18]:
cms_knn_bf = confusion_matrix(y_true=y_test_kt.to_numpy().ravel(), y_pred=y_pred_knn)
cms_knn_bf

array([[98, 19],
       [17, 90]], dtype=int64)

#### Stablo odluke

In [19]:
# Hiperparametri stabla odluke
params_dt = [{
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'ccp_alpha': [i/10 for i in range(1, 4)],
    'max_depth': [i for i in range(3, 6)]
}]

gs_dt = GridSearchCV(estimator=model_dt, param_grid=params_dt, cv=8).fit(X=X_train_kt.to_numpy(), y=y_train_kt.to_numpy().ravel())

Najbolji parametri modela stabla odlučivanja su

In [20]:
gs_dt.best_params_

{'ccp_alpha': 0.1, 'criterion': 'gini', 'max_depth': 3, 'splitter': 'best'}

In [21]:
model_dt.criterion = gs_dt.best_params_['criterion']
model_dt.splitter = gs_dt.best_params_['splitter']
model_dt.ccp_alpha = gs_dt.best_params_['ccp_alpha']
model_dt.max_depth = gs_dt.best_params_['max_depth']

model_dt.fit(X=X_train_kt.to_numpy(), y=y_train_kt.to_numpy().ravel())
y_pred_dt = model_dt.predict(X=X_test_kt.to_numpy())

In [22]:
as_dt_bf = accuracy_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=y_pred_dt)
as_dt_bf

0.7991071428571429

In [23]:
ps_dt_bf = precision_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=y_pred_dt)
ps_dt_bf

0.7627118644067796

In [24]:
rs_dt_bf = recall_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=y_pred_dt)
rs_dt_bf

0.8411214953271028

In [25]:
f1s_dt_bf = f1_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=y_pred_dt)
f1s_dt_bf

0.7999999999999999

In [26]:
cms_dt_bf = confusion_matrix(y_true=y_test_kt.to_numpy().ravel(), y_pred=y_pred_dt)
cms_dt_bf

array([[89, 28],
       [17, 90]], dtype=int64)

Sva tri modela daju odlične rezultate pri testiranju, te ne ukazuju na *overfit* nad podacima.

Problem klasifikacije koji rješavamo zahtjeva dobre karakteristike jer je u pitanju ogromna težina problema - nemamo prostora za greške kada se radi o predviđanju toga da li pacijent ima srčanu bolest ili ne. Stoga, potrebno je da naš model pri greškama daje više ***false positive*** vrijednosti no ***false negative*** jer *biće **gora** stvar ako nekoga ko ima srčanu bolest nažalost otpišemo jer naš model predviđa da je nema (**false negative**) nego da uključimo u liječenje i one koji zapravo nemaju i kod kojih će se kasnije zaista ponovnim analizama utvrditi da nemaju (**false positive**).*

Iako želimo da postignemo savršenu "preciznost" (u kontekstu da nemamo ovakvih grešaka pri predviđanjima), naši modeli svakako pokazuju dobru karakteristiku da postoji više *false positive* vrijednosti no *false negative*.

### Smanjenje dimenzionalnosti problema

Iako smo u pretprocesiranju podataka pokušali smanjiti dimenzionalnost problema određivanjem koleracije kako između *feature*-a i *target*-a, tako i između *feature*-a međusobno, sada ćemo pokušati drugom tehnikom da uvidimo da li možemo izbaciti određene *feature*-e ali opet zadržati dobre performanse modela. Tehnika koju ćemo koristiti jeste *SelectKBest*.

Naš model posjeduje 19 *feature*-a trenutno, a mi ćemo za svaki model arbitrarno pokušati smanjiti ih na 10. Kasnije ćemo uporediti dobijene performanse i vidjeti u kojem stepenu smo ih uspjeli održati i nakon izbacivanja nekoliko *feature*-a.

Kreiraćemo nove modele i uporediti njihove performanse sa starim. Naravno, preuzećemo prethodno nađene optimalne hiperparametre starih modela i unijeti ih u nove.

In [27]:
new_model_lr = LogisticRegression()
new_model_knn = KNeighborsClassifier()
new_model_dt = DecisionTreeClassifier()

# Preuzimanje hiperparametara modela logističke regresije
new_model_lr.C = gs_lr.best_params_['C']
new_model_lr.tol = gs_lr.best_params_['tol']
new_model_lr.solver = gs_lr.best_params_['solver']

# Preuzimanje hiperparametara KNN modela
new_model_knn.n_neighbors = gs_knn.best_params_['n_neighbors']
new_model_knn.weights = gs_knn.best_params_['weights']
new_model_knn.algorithm = gs_knn.best_params_['algorithm']
new_model_knn.metric = gs_knn.best_params_['metric']

# Preuzimanje hiperparametara modela stabla odluke
new_model_dt.criterion = gs_dt.best_params_['criterion']
new_model_dt.splitter = gs_dt.best_params_['splitter']
new_model_dt.ccp_alpha = gs_dt.best_params_['ccp_alpha']
new_model_dt.max_depth = gs_dt.best_params_['max_depth']

#### Logistička regresija

In [28]:
from sklearn.feature_selection import SelectKBest, f_classif

k_best_lr = SelectKBest(score_func=f_classif, k=10)
k_best_lr.fit_transform(X=X_train_lr.to_numpy(), y=y_train_lr.to_numpy().ravel())

new_X_train_lr = X_train_lr.iloc[:, k_best_lr.get_support(indices=True)]
new_X_test_lr = X_test_lr.iloc[:, k_best_lr.get_support(indices=True)]

new_X_train_lr.head()


,Age,RestingBP,MaxHR,Oldpeak,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,-0.500000,0.4,0.444444,-0.333333,True,False,True,False,False,True
1,0.071429,-0.1,-0.277778,1.000000,False,False,False,True,True,False
2,1.571429,-1.3,-0.555556,2.000000,True,False,True,False,False,False
3,0.642857,0.5,1.083333,-0.333333,False,True,False,False,False,True
4,0.785714,-0.5,0.000000,-0.066667,True,False,False,False,False,True


In [29]:
new_model_lr.fit(X=new_X_train_lr.to_numpy(), y=y_train_lr.to_numpy().ravel())
new_y_pred_lr = new_model_lr.predict(X=new_X_test_lr.to_numpy())

Uporedimo performanse starog modela sa više *feature*-a i novog modela sa manje pomenutih tako što ćemo oduzeti prethodne performanse od trenutnih. Ako dobijemo negativnu razliku, znači da je model poboljšao performanse - u suprotnom, "gore" su.

In [30]:
as_lr_bf - accuracy_score(y_true=y_test_lr.to_numpy().ravel(), y_pred=new_y_pred_lr)

0.008928571428571397

In [31]:
ps_lr_bf - precision_score(y_true=y_test_lr.to_numpy().ravel(), y_pred=new_y_pred_lr)

0.01650943396226412

In [32]:
rs_lr_bf - recall_score(y_true=y_test_lr.to_numpy().ravel(), y_pred=new_y_pred_lr)

0.0

In [33]:
f1s_lr_bf - f1_score(y_true=y_test_lr.to_numpy().ravel(), y_pred=new_y_pred_lr)

0.008099147809447538

In [34]:
cms_lr_bf - confusion_matrix(y_true=y_test_lr.to_numpy().ravel(), y_pred=new_y_pred_lr)

array([[ 2, -2],
       [ 0,  0]], dtype=int64)

Očigledno je da su razlike performansi gotovo nepostojeće te ne gubimo validnost modela pri smanjenju dimenzionalnosti. Provjerimo kakav je slučaj kod KNN modela, kao i stabla odlučivanja.

#### KNN model i stablo odlučivanja

In [35]:
k_best_kt = SelectKBest(score_func=f_classif, k=10)
k_best_kt.fit_transform(X=X_train_kt.to_numpy(), y=y_train_kt.to_numpy().ravel())

new_X_train_kt = X_train_kt.iloc[:, k_best_kt.get_support(indices=True)]
new_X_test_kt = X_test_kt.iloc[:, k_best_kt.get_support(indices=True)]

new_X_train_kt.head()

,Age,MaxHR,Oldpeak,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,0.785714,0.210526,0.1875,False,False,False,True,False,False,True
1,-0.071429,-0.342105,-0.3125,False,False,True,True,False,False,True
2,0.142857,-0.315789,0.3125,True,False,False,False,True,False,True
3,0.857143,-0.052632,1.1250,True,True,False,True,False,False,True
4,0.857143,0.289474,-0.0625,True,True,False,True,False,True,False


In [36]:
new_model_knn.fit(X=new_X_train_kt.to_numpy(), y=y_train_kt.to_numpy().ravel())
new_model_dt.fit(X=new_X_train_kt.to_numpy(), y=y_train_kt.to_numpy().ravel())

new_y_pred_knn = new_model_knn.predict(X=new_X_test_kt.to_numpy())
new_y_pred_dt = new_model_dt.predict(X=new_X_test_kt.to_numpy())

KNN model

In [37]:
as_knn_bf - accuracy_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=new_y_pred_knn)

0.0267857142857143

In [38]:
ps_knn_bf - precision_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=new_y_pred_knn)

0.03807745392546891

In [39]:
rs_knn_bf - recall_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=new_y_pred_knn)

0.009345794392523366

In [40]:
f1s_knn_bf - f1_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=new_y_pred_knn)

0.024242424242424288

In [41]:
cms_knn_bf - confusion_matrix(y_true=y_test_kt.to_numpy().ravel(), y_pred=new_y_pred_knn)

array([[ 5, -5],
       [-1,  1]], dtype=int64)

Model stabla odluke

In [42]:
as_dt_bf - accuracy_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=new_y_pred_dt)

0.0

In [43]:
ps_dt_bf - precision_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=new_y_pred_dt)

0.0

In [44]:
rs_dt_bf - recall_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=new_y_pred_dt)

0.0

In [45]:
f1s_dt_bf - f1_score(y_true=y_test_kt.to_numpy().ravel(), y_pred=new_y_pred_dt)

0.0

In [46]:
cms_dt_bf - confusion_matrix(y_true=y_test_kt.to_numpy().ravel(), y_pred=new_y_pred_dt)

array([[0, 0],
       [0, 0]], dtype=int64)

Za svaki model dobijamo da su maksimalna odstupanja u rangu od &plusmn;3%, što znači da smo uspjeli smanjiti dimenzionalnost problema bez značajnog gubitka na performansama. Korisniku ostavljamo da pokuša da još više degradira dimenzionalnost te uporedi performanse.

### Finalni zaključak

Modeli koji se najuspješnije pokazuju kroz mnogostruka testiranja jesu model logističke regresije, kao i KNN model. Generalno se najlošije pokazuje model stabla odluke, ali ne za ogroman gubitak performansi. Zaključak jeste da modeli mogu vrlo dobro (do 90% preciznosti, *napomena da preciznost nije uvijek dobar indikator dobrog modela*) opisati sistem i predvidjeti da li neko ima srčanu bolest, uz mali procenat pogrešnih procjena. 